Created on Friday Feb  3 10:04:17 2023
@author: Ioannis Matthaiou, Research Fellow, University of Southampton

Load packages

In [ ]:
#% Load Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import xgboost as xgb
import sklearn.preprocessing
import sklearn.feature_selection
import sklearn.ensemble
import scipy.stats
import seaborn as sns
import os
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFECV, SelectFromModel
from sklearn.model_selection import cross_val_predict, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix
import xgboost
from sklearn.neighbors import KNeighborsClassifier

%matplotlib inline

Exploratory data analysis
1. Load data
2. Assign to DFs

In [ ]:
plt.style.use('ggplot')
parent_dir = './'

df_train = pd.read_csv(parent_dir + './Exercise data exploration/data/final/train.csv') # Data frame of training set
df_test = pd.read_csv(parent_dir + './Exercise data exploration/data/final/test.csv') # Data frame of testing set

Ntrain, ptrain = df_train.shape # get dimension of training set
Ntest, ptest = df_test.shape # get dimension of testing set
ptrain = ptrain-2
ptest = ptest-2
print('Number of features = ' + str(ptrain))
print('Number of training samples = ' + str(Ntrain))
print('Number of testing samples = ' + str(Ntest))
if ptrain != ptest:
    print('Number of columns differ on testing and training sets by ', np.abs(ptest-ptrain))

pd.options.display.max_columns = None
print('Training set first few rows')
print('---------------------------')
print(df_train.head(3)) # Load first 3 rows of the training set
print('Testing set first few rows')
print('---------------------------')
print(df_test.head(3)) # Load first 3 rows of the training set
# drop datasetId as it is a constant
df_train = df_train.drop(columns=["datasetId"])
df_test = df_test.drop(columns=["datasetId"])

Datasets summary statistics and fractions of each class:
1. Dataset may be slighty imbalanced in terms of classes
2. Very different scales of each feature 

In [ ]:
print('Training set summary statistics')
print('-------------------------------')
print(df_train.describe()) # Descriptive statistics in training set
print('Testing set summary statistics')
print('-------------------------------')
print(df_test.describe()) # Descriptive statistics in training set

print('Ratio of test set to train set is ' + str(np.round(len(df_test)/len(df_train),decimals=2)))

Ncntstrain = df_train['condition'].value_counts()
Ncntstest = df_test['condition'].value_counts()
fig, (ax1, ax2) = plt.subplots(2, 1)
df_train.condition.value_counts(normalize=True).sort_values().plot(kind = 'barh', ax=ax1)
ax1.set_title('Fraction of samples per each condition in training set')
df_test.condition.value_counts(normalize=True).sort_values().plot(kind = 'barh', ax=ax2)
ax2.set_title('Fraction of samples per each condition in testing set')
fig.tight_layout()
plt.show()

# Checks if missing values are present
if df_train.isnull().any().any() == True:
    print('Missing values in training set')   
if df_test.isnull().any().any() == True:
    print('Missing values in testing set')

Visualise data distributions by plotting four different features:
1. It is clear that Gaussian distribution assumption is violated in most of the cases
2. Amount of bandwidth in kernel density estimates is quite important to avoid oversmoothing

In [ ]:
kdebw = 3
fig, axes = plt.subplots(nrows=1,ncols=4,sharey=False,figsize = (12, 3))         
sns.kdeplot(data=df_train,x=df_train['RMSSD'],
                        fill=True, common_norm=False, ax=axes[0],alpha=.25, 
                        bw_method='scott',bw_adjust=kdebw,legend=True) 
sns.kdeplot(data=df_train,x=df_train['MEDIAN_RR'],
                        fill=True, common_norm=False, ax=axes[1],alpha=.25, 
                        bw_method='scott',bw_adjust=kdebw,legend=True) 
sns.kdeplot(data=df_train,x=df_train['LF_HF'],
                        fill=True, common_norm=False, ax=axes[2],alpha=.25, 
                        bw_method='scott',bw_adjust=kdebw,legend=True)   
sns.kdeplot(data=df_train,x=df_train['VLF'],
                        fill=True, common_norm=False, ax=axes[3],alpha=.25, 
                        bw_method='scott',bw_adjust=kdebw,legend=True)                  
fig.tight_layout()
plt.show()

Scale data, so that each feature is on the same scale:
1. Scale features by z-score standardisation if their close approximation to Gaussian dist.
2. Min Max is a more general scaling technique that is also not influenced by outliers
Encode labels into nominal variables, so that it is compatible with most classification algorithm implementations

In [ ]:
scaler = sklearn.preprocessing.MinMaxScaler()
# transformed the features
X_train = scaler.fit_transform(df_train[df_train.columns[:ptrain]].to_numpy())
df_train_sc = pd.DataFrame(X_train, columns=df_train.columns[:ptrain])
print("Normalised Features:\n", df_train_sc[:3])
df_train_sc = pd.concat([df_train_sc, df_train['condition']], 1)
labenc = sklearn.preprocessing.LabelEncoder()
y_train = labenc.fit_transform(df_train['condition'])

X_test = scaler.transform(df_test[df_test.columns[:ptest]].to_numpy())
y_test = labenc.fit_transform(df_test['condition'])

Run boxplotsub function to be used later on (twice)

In [ ]:
def boxplotsub(dframe):      
    k2=17
    nrows = 2
    ncols = 1
    fig, axes = plt.subplots(nrows=nrows,ncols=ncols,sharey=False,figsize = (10, 25)) 
    # Feature batch kk:
    for i in range(nrows):          
        print(range((i * k2),(i * k2) + k2,1))
        df_tmp = dframe[dframe.columns[range((i * k2),(i * k2) + k2,1)]] 
        sns.boxplot(data=df_tmp, ax=axes[i],
                    showmeans=True, meanprops={"marker":"o",
                                               "markerfacecolor":"white", 
                                               "markeredgecolor":"black",
                                               "markersize":"10"})   
        axes[i].tick_params(labelrotation=90)        
    plt.rcParams['font.size'] = 18
    fig.tight_layout()    
    plt.show()

Visualise features better by looking at their scaled versions (compare with the original box plots):
1. Many of the features have extreme values and thus are highly skewed
2. Can make attempt to remove them if and only if classification accuracy seems to be suffering

In [ ]:
boxplotsub(df_train)
boxplotsub(df_train_sc)

In [ ]:
lo_perctl = .1
hi_perctl = .9
remvout = 0 # 1: remove/replace outliers, 0: not remove/replace outliers
def uvoutlremv(dfc,loper,hiper,chk):
    q1 = dfc.quantile(loper)
    q3 = dfc.quantile(hiper)
    iqr = q3-q1
    loval = q1-iqr
    hival = q3+iqr
    print('Skewness before outlier replacement: ', np.round(dfc.skew(),1))
    print('Maximum value before outlier replacement: ', np.round(dfc.max(),1))  
    print('Minimum value before outlier replacement: ', np.round(dfc.min(),1))  
    if chk==1:
        print('Number of outliers removed: ', len(dfc.loc[(dfc<loval) | (dfc>hival)]))
        # dfc.loc[(dfc<loval) | (dfc>hival)] = dfc.median()      
        dfc.loc[(dfc<loval) | (dfc>hival)] = np.nan
        print('Skewness after outlier replacement: ', np.round(dfc.skew(),1)) 
        print('Maximum value after outlier replacement: ', np.round(dfc.max(),1))  
        print('Minimum value after outlier replacement: ', np.round(dfc.min(),1))
    else:
        print('Number of outliers found: ', len(dfc.loc[(dfc<loval) | (dfc>hival)]))         
    return dfc

df_train['VLF'] = uvoutlremv(dfc=df_train['VLF'],loper=lo_perctl,hiper=hi_perctl,chk=remvout)
df_train['SD2'] = uvoutlremv(dfc=df_train['SD2'],loper=lo_perctl,hiper=hi_perctl,chk=remvout)
df_train['MEAN_RR'] = uvoutlremv(dfc=df_train['MEAN_RR'],loper=lo_perctl,hiper=hi_perctl,chk=remvout)
df_train['MEDIAN_RR'] = uvoutlremv(dfc=df_train['MEDIAN_RR'],loper=lo_perctl,hiper=hi_perctl,chk=remvout)
df_train['TP'] = uvoutlremv(dfc=df_train['TP'],loper=lo_perctl,hiper=hi_perctl,chk=remvout)
df_train['KURT'] = uvoutlremv(dfc=df_train['KURT'],loper=lo_perctl,hiper=hi_perctl,chk=remvout)
df_train['LF_HF'] = uvoutlremv(dfc=df_train['LF_HF'],loper=lo_perctl,hiper=hi_perctl,chk=remvout)
df_train_sc = df_train_sc.dropna(axis = 0)
print('-------OUTLIERS REMOVED : ', len(df_train)-len(df_train_sc))

Rescale and replot the distribution of classes if remvout = 1 and reassign to arrays

In [ ]:
if remvout==1:
    boxplotsub(df_train_sc)
    # transformed the feature
    X_train = scaler.fit_transform(df_train[df_train.columns[:ptrain]].to_numpy())
    df_train_sc = pd.DataFrame(X_train, columns=df_train.columns[:ptrain])
    print("Normalised Features:\n", df_train_sc[:3])
    df_train_sc = pd.concat([df_train_sc, df_train['condition']], 1)
    labenc = sklearn.preprocessing.LabelEncoder()
    y_train = labenc.fit_transform(df_train['condition'])
    Ncntstrain = df_train_sc['condition'].value_counts()
    df_train_sc.condition.value_counts(normalize=True).sort_values().plot(kind = 'barh')
    ax1.set_title('Fraction of samples per each condition in training set')
    fig.tight_layout()
    plt.show()
    
    y_train = labenc.fit_transform(df_train_sc['condition'])
    X_train = df_train_sc[df_train_sc.columns[:ptrain]].to_numpy()

Density plots to summarise data in terms of their distributional properties:
1. Plot KDE of each feature conditioned on class labels ('condition')
2. Check how each feature is distributed when conditioned on the class label and whether there is any change
3. It is clear that features are sensitive to the class - either being shifted in x or its peak value changes
4. Some of the features conform to a power law: may need to log-transform them

In [ ]:
k=0
nrows = 3
ncols = 3
Nkdeplots = int(np.ceil(ptrain/(nrows*ncols)))

for kk in range(Nkdeplots):
    fig, axes = plt.subplots(nrows=nrows,ncols=ncols,sharey=False,figsize = (30, 25))
    # Feature batch kk:
    for i in range(nrows):
        if kk == 3 and i == 2:
            ncols = ncols-2
        for j in range(ncols):            
            sns.kdeplot(data=df_train_sc,x=df_train_sc.columns[k],hue='condition',
                        fill=True, common_norm=False, ax=axes[i,j],alpha=.25, 
                        bw_method='scott',bw_adjust=kdebw,legend=True)   
            k += 1          
    fig.tight_layout()
plt.show()

View feature-feature linear correlation
1. Add a threshold to check only those that are significant
2. If needed we can run correlation function to remove the most correlated features

In [ ]:
# {‘pearson’, ‘kendall’, ‘spearman’}
# pearson : standard correlation coefficient
# kendall : Kendall Tau correlation coefficient
# spearman : Spearman rank correlation
cut_off = 0.9
corrmat = df_train_sc[df_train_sc.columns[:ptrain]].corr(method='pearson')
sns.set_theme(style="white")
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corrmat, dtype=bool))
mask |= np.abs(corrmat) < cut_off
corrmat = corrmat[~mask]
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corrmat, mask=mask, cmap='BrBG', vmin=-1, vmax=1,
            square=True, linewidths=.5, cbar_kws={"shrink": .25})
plt.show()

# Feature selection
# Script that can be used to remove a feature that is correlated with any other feature
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr(method='pearson')
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr
# df_train_sc.drop(corr_features,axis=1) # do the same for the testing set

corr_features = correlation(df_train_sc, cut_off)
print('Highly correlated set of features is: ', corr_features,' according to cut_off of ',cut_off)

Feature selection by means of:
1. Anova F-test (Linear and Gaussian distributed data)
2. Mutual information (Does not assume linearity nor Gaussianity)
3. Random Forest classifier via Mean Impurity Decrease (Tied to the method)
4. XGBoost classifier via Built-in Feature Importance (Tied to the method)

Generally, HR is found to be a good feature that discriminates between the three classes. While ANOVA F-test and Mean Impurity Decrese from the Random Forest yield similarities, e.g. MEAN_RR is found by both as being important or statistically significant (in the F-test scenario).
Although in features are linearly correlated (e.g. MEAN_RR - MEDIAN_RR), as seen above, a Random Forest or XGBoost algorithm assigns feature importance, so that it also automatically deals with the problem of feature selection. That is of course not to say that these algorithms can deal with a large number of correlated variables.

In [ ]:
# Mutual information
mi_score = sklearn.feature_selection.mutual_info_classif(X_train,y_train)
mi_score /= mi_score.max()
# ANOVA 
Fscore,Fpval = sklearn.feature_selection.f_classif(X_train,y_train)
Fscore /= Fscore.max()
#Random forest feature importance
Ntrees = 25
modelRF = sklearn.ensemble.RandomForestClassifier(n_estimators = Ntrees, n_jobs=-1)
modelRF.fit(X_train, y_train)
# get importance
RFscore = modelRF.feature_importances_
RFscore /= RFscore.max()
# XGBoost feature importance
modelXGB = xgboost.XGBClassifier(n_estimators = Ntrees, n_jobs=-1)
modelXGB.fit(X_train, y_train)
# get importance
XGBscore = modelXGB.feature_importances_
XGBscore /= XGBscore.max()

# Plot scores of feature selection
df_FSscores=pd.DataFrame({'Mutual Inf':mi_score,
                          'F Score':Fscore,
                          'RF Score':RFscore,
                          'XGB Score':XGBscore,
                          'Feature':df_train_sc.columns[:ptrain]})
df_FSscores.set_index('Feature', inplace = True)
df_FSscores.sort_values('XGB Score', inplace = True, ascending = False)
fig, ax = plt.subplots(ncols=1,nrows=4,sharey=True,sharex=True,figsize = (10, 15))
sns.lineplot(x='Feature', y='Mutual Inf', data=df_FSscores, ax=ax[0])
sns.lineplot(x='Feature', y='F Score', data=df_FSscores, ax=ax[1])
sns.lineplot(x='Feature', y='RF Score', data=df_FSscores, ax=ax[2])
sns.lineplot(x='Feature', y='XGB Score', data=df_FSscores, ax=ax[3])
plt.xticks(rotation='vertical')
plt.grid(visible=True,which='major',axis='y')
fig.tight_layout()
plt.show()

Make predictions on the test data

In [ ]:
modelRF.fit(X_train, y_train)
yhat = modelRF.predict(X_test)
accuracy = accuracy_score(y_test, yhat)
print("Accuracy in test set = ", (accuracy * 100.0), " %")

Compare with the simplest classifier - KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=15)
kfold = StratifiedKFold(n_splits=5, shuffle=True)
yhat = cross_val_predict(knn, X_train, y_train, cv=kfold)
print("Accuracy in training set = ", (accuracy_score(y_train, yhat) * 100.0), " %")
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)
yhat = knn.predict(X_test)
print("Accuracy in test set = ", (accuracy_score(y_test, yhat) * 100.0), " %")

Test decrease in classification accuracy by removing important features computed by Random Forest :
As can be seen we can still have close to 100% accuracy on the test set even with 7 or 8 features, i.e. the top ones selected by the Random Forest.

In [ ]:
featsimportances = np.sort(modelRF.feature_importances_)
for rfimp in featsimportances:
	# select features from model
	FS_sel = SelectFromModel(modelRF, threshold=rfimp, prefit=True)
	X_train_red = FS_sel.transform(X_train)
	# classification model training
	MODEL_C = xgboost.XGBClassifier(n_estimators = Ntrees, n_jobs=-1)
	MODEL_C.fit(X_train_red, y_train)
	# classification model evaluation
	X_test_red = FS_sel.transform(X_test)
	yhat = MODEL_C.predict(X_test_red)
	print('Trained with =',X_train_red.shape[1], 'number of features')
	print('Threshold=',rfimp, 'Accuracy=', accuracy_score(y_test, yhat)*100.0,'%')